# Basic tour of the Bayesian Optimization package

This is a constrained global optimization package built upon bayesian inference and gaussian process, that attempts to find the maximum value of an unknown function in as few iterations as possible. This technique is particularly suited for optimization of high cost functions, situations where the balance between exploration and exploitation is important.

Bayesian optimization works by constructing a posterior distribution of functions (gaussian process) that best describes the function you want to optimize. As the number of observations grows, the posterior distribution improves, and the algorithm becomes more certain of which regions in parameter space are worth exploring and which are not, as seen in the picture below.

As you iterate over and over, the algorithm balances its needs of exploration and exploitation taking into account what it knows about the target function. At each step a Gaussian Process is fitted to the known samples (points previously explored), and the posterior distribution, combined with a exploration strategy (such as UCB (Upper Confidence Bound), or EI (Expected Improvement)), are used to determine the next point that should be explored (see the gif below).

This process is designed to minimize the number of steps required to find a combination of parameters that are close to the optimal combination. To do so, this method uses a proxy optimization problem (finding the maximum of the acquisition function) that, albeit still a hard problem, is cheaper (in the computational sense) and common tools can be employed. Therefore Bayesian Optimization is most adequate for situations where sampling the function to be optimized is a very expensive endeavor. See the references for a proper discussion of this method.

## 1. Specifying the function to be optimized

This is a function optimization package, therefore the first and most important ingredient is, of course, the function to be optimized.

**DISCLAIMER:** We know exactly how the output of the function below depends on its parameter. Obviously this is just an example, and you shouldn't expect to know it in a real scenario. However, it should be clear that you don't need to. All you need in order to use this package (and more generally, this technique) is a function `f` that takes a known set of parameters and outputs a real number.

In [2]:
def black_box_function(x, y):
    """Function with unknown internals we wish to maximize.

    This is just serving as an example, for all intents and
    purposes think of the internals of this function, i.e.: the process
    which generates its output values, as unknown.
    """
    return -x ** 2 - (y - 1) ** 2 + 1

## 2. Getting Started

All we need to get started is to instantiate a `BayesianOptimization` object specifying a function to be optimized `f`, and its parameters with their corresponding bounds, `pbounds`. This is a constrained optimization technique, so you must specify the minimum and maximum values that can be probed for each parameter in order for it to work

In [3]:
from bayes_opt import BayesianOptimization

In [4]:
# Bounded region of parameter space
pbounds = {'x': (2, 4), 'y': (-3, 3)}

In [5]:
optimizer = BayesianOptimization(
    f=black_box_function,
    pbounds=pbounds,
    verbose=2, # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent
    random_state=1,
)

The BayesianOptimization object will work out of the box without much tuning needed. The main method you should be aware of is `maximize`, which does exactly what you think it does.

There are many parameters you can pass to maximize, nonetheless, the most important ones are:
- `n_iter`: How many steps of bayesian optimization you want to perform. The more steps the more likely to find a good maximum you are.
- `init_points`: How many steps of **random** exploration you want to perform. Random exploration can help by diversifying the exploration space.

In [6]:
optimizer.maximize(
    init_points=2,
    n_iter=3,
)

|   iter    |  target   |     x     |     y     |
-------------------------------------------------
| 1         | -7.135    | 2.8340440 | 1.3219469 |
| 2         | -7.78     | 2.0002287 | -1.186004 |
| 3         | -7.157    | 2.8375977 | 1.3238498 |
| 4         | -6.633    | 2.7487090 | 1.2790562 |
| 5         | -5.751    | 2.5885326 | 1.2246876 |


The best combination of parameters and target value found can be accessed via the property `bo.max`.

In [7]:
print(optimizer.max)

{'target': np.float64(-5.750985875689304), 'params': {'x': np.float64(2.5885326650623566), 'y': np.float64(1.2246876000015976)}}


While the list of all parameters probed and their corresponding target values is available via the property `bo.res`.

In [8]:
for i, res in enumerate(optimizer.res):
    print("Iteration {}: \n\t{}".format(i, res))

Iteration 0: 
	{'target': np.float64(-7.135455292718879), 'params': {'x': np.float64(2.8340440094051482), 'y': np.float64(1.3219469606529486)}}
Iteration 1: 
	{'target': np.float64(-7.779531005607566), 'params': {'x': np.float64(2.0002287496346898), 'y': np.float64(-1.1860045642089614)}}
Iteration 2: 
	{'target': np.float64(-7.156839989425082), 'params': {'x': np.float64(2.8375977943744273), 'y': np.float64(1.3238498831039895)}}
Iteration 3: 
	{'target': np.float64(-6.633273772355583), 'params': {'x': np.float64(2.7487090390562576), 'y': np.float64(1.2790562505410115)}}
Iteration 4: 
	{'target': np.float64(-5.750985875689304), 'params': {'x': np.float64(2.5885326650623566), 'y': np.float64(1.2246876000015976)}}


### 2.1 Changing bounds

During the optimization process you may realize the bounds chosen for some parameters are not adequate. For these situations you can invoke the method `set_bounds` to alter them. You can pass any combination of **existing** parameters and their associated new bounds.

In [9]:
optimizer.set_bounds(new_bounds={"x": (-2, 3)})

In [10]:
optimizer.maximize(
    init_points=0,
    n_iter=5,
)

|   iter    |  target   |     x     |     y     |
-------------------------------------------------
| 6         | -4.438    | 2.3269441 | 1.1533794 |
| 7         | -2.42     | 1.8477442 | 0.9230233 |
| 8         | -0.2088   | 1.0781674 | 1.2152869 |
| 9         | 0.7797    | -0.298812 | 1.3619705 |
| 10        | -3.391    | -0.655060 | 2.9904883 |


## 3. Guiding the optimization

It is often the case that we have an idea of regions of the parameter space where the maximum of our function might lie. For these situations the `BayesianOptimization` object allows the user to specify specific points to be probed. By default these will be explored lazily (`lazy=True`), meaning these points will be evaluated only the next time you call `maximize`. This probing process happens before the gaussian process takes over.

Parameters can be passed as dictionaries such as below:

In [11]:
optimizer.probe(
    params={"x": 0.5, "y": 0.7},
    lazy=True,
)

Or as an iterable. Beware that the order has to match the order of the initial `pbounds` dictionary. You can usee `optimizer.space.keys` for guidance

In [12]:
print(optimizer.space.keys)

['x', 'y']


In [13]:
optimizer.probe(
    params=[-0.3, 0.1],
    lazy=True,
)

In [14]:
optimizer.maximize(init_points=0, n_iter=0)

|   iter    |  target   |     x     |     y     |
-------------------------------------------------
| 11        | 0.66      | 0.5       | 0.7       |
| 12        | 0.1       | -0.3      | 0.1       |


## 4. Saving, loading and restarting

By default you can follow the progress of your optimization by setting `verbose>0` when instantiating the `BayesianOptimization` object. If you need more control over logging/alerting you will need to use an observer. For more information about observers checkout the advanced tour notebook. Here we will only see how to use the native `JSONLogger` object to save to and load progress from files.

### 4.1 Saving progress

In [15]:
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events

The observer paradigm works by:
1. Instantiating an observer object.
2. Tying the observer object to a particular event fired by an optimizer.

The `BayesianOptimization` object fires a number of internal events during optimization, in particular, every time it probes the function and obtains a new parameter-target combination it will fire an `Events.OPTIMIZATION_STEP` event, which our logger will listen to.

**Caveat:** The logger will not look back at previously probed points.

In [16]:
logger = JSONLogger(path="./logs.log")
optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)

In [17]:
optimizer.maximize(
    init_points=2,
    n_iter=3,
)

|   iter    |  target   |     x     |     y     |
-------------------------------------------------
| 13        | -2.96     | -1.989407 | 0.9536339 |
| 14        | -0.7135   | 1.0509704 | 1.7803462 |
| 15        | -18.33    | -1.976933 | -2.927535 |
| 16        | 0.9097    | -0.228312 | 0.8046706 |
| 17        | 0.913     | 0.2069253 | 1.2101397 |


### 4.2 Loading progress

Naturally, if you stored progress you will be able to load that onto a new instance of `BayesianOptimization`. The easiest way to do it is by invoking the `load_logs` function, from the `util` submodule.

In [18]:
from bayes_opt.util import load_logs

In [19]:
new_optimizer = BayesianOptimization(
    f=black_box_function,
    pbounds={"x": (-2, 2), "y": (-2, 2)},
    verbose=2,
    random_state=7,
)
print(len(new_optimizer.space))

0


In [20]:
load_logs(new_optimizer, logs=["./logs.log"]);

/Users/adrianmolzon/oss/BayesianOptimization/bayes_opt/bayesian_optimization.py:240: UserWarning: 
Data point [-1.97693359 -2.9275359 ] is outside the bounds of the parameter y.
	Bounds:
[-2.  2.]
  self._space.register(params, target, constraint_value)


In [21]:
print("New optimizer is now aware of {} points.".format(len(new_optimizer.space)))

New optimizer is now aware of 5 points.


In [22]:
new_optimizer.maximize(
    init_points=0,
    n_iter=10,
)

|   iter    |  target   |     x     |     y     |
-------------------------------------------------
| 1         | -7.101    | 1.9973177 | -1.027773 |
| 2         | -0.6311   | -0.806373 | 1.9903823 |
| 3         | 0.2572    | 0.6921933 | 0.4865326 |
| 4         | -3.0      | 1.9990210 | 0.9362987 |
| 5         | 0.9011    | 0.1556921 | 0.7268099 |
| 6         | 0.9082    | -0.238045 | 1.1874540 |
| 7         | 0.9988    | 0.0204153 | 0.9723584 |
| 8         | 0.8702    | 0.3580753 | 0.9606191 |
| 9         | 0.997     | -0.046534 | 1.0287398 |
| 10        | 0.9979    | 0.0295378 | 1.0346895 |


## 5. Saving and loading the optimizer state

The optimizer state can be saved to a file and loaded from a file. This is useful for continuing an optimization from a previous state, or for analyzing the optimization history without running the optimizer again.

### 5.1 Saving the optimizer state

The optimizer state can be saved to a file using the `save_state` method.
optimizer.save_state("./optimizer_state.json")

In [23]:
optimizer.save_state("optimizer_state.json")

## 5.2 Loading the optimizer state

There are two ways to load a saved state, depending on your needs:

1. With the target function (to continue optimization):


In [24]:
new_optimizer = BayesianOptimization.load(
    "optimizer_state.json",
    f=black_box_function
)

# Continue optimization
new_optimizer.maximize(
    init_points=0,
    n_iter=5
)

|   iter    |  target   |     x     |     y     |
-------------------------------------------------
| 1         | 0.9988    | 0.0196213 | 0.9714562 |
| 2         | 0.9214    | 0.0346617 | 0.7218000 |
| 3         | 0.9767    | -0.116593 | 1.0987579 |
| 4         | 0.9967    | 0.0414245 | 1.0402841 |
| 5         | 0.985     | 0.1168371 | 0.9630685 |


2. Without the target function (for analysis only):


In [25]:
# Load state for analysis
analysis_optimizer = BayesianOptimization.load("optimizer_state.json")

# Can inspect results
print("Best result:", analysis_optimizer.max)
print("All results:", len(analysis_optimizer.res), "points")

Best result: {'target': np.float64(0.913023173060441), 'params': {'x': np.float64(0.20692536368024994), 'y': np.float64(1.2101397649312364)}}
All results: 17 points


This provides a simpler alternative to the logging system shown in section 4, especially when you want to continue optimization from a previous state.

## Next Steps

This tour should be enough to cover most usage scenarios of this package. If, however, you feel like you need to know more, please checkout the `advanced-tour` notebook. There you will be able to find other, more advanced features of this package that could be what you're looking for. Also, browse the examples folder for implementation tips and ideas.